# Attention

- toc: true 
- badges: true
- comments: true
- sticky_rank: 3
- author: Bowen
- categories: [pytorch, fastai]

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Softmax
from fastcore.foundation import *
from fastcore.utils import *
from fastcore.test import *
from nbdev.showdoc import *
from fastcore.dispatch import typedispatch
from functools import partial
import numpy as np
import inspect
from torch.nn.functional import softmax

## CCNet

![](https://i.loli.net/2020/09/15/t9KXn8cu51wDsiG.png)

In [2]:
def INF(B,H,W):
     return -torch.diag(torch.tensor(float("inf")).cuda().repeat(H),0).unsqueeze(0).repeat(B*W,1,1)

why use contiguous method in pytorch? https://zhuanlan.zhihu.com/p/64551412

In [3]:
aa = torch.arange(300).reshape(12,5,5)
aa
aa.stride()
aa.stride(1)

tensor([[[  0,   1,   2,   3,   4],
         [  5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14],
         [ 15,  16,  17,  18,  19],
         [ 20,  21,  22,  23,  24]],

        [[ 25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39],
         [ 40,  41,  42,  43,  44],
         [ 45,  46,  47,  48,  49]],

        [[ 50,  51,  52,  53,  54],
         [ 55,  56,  57,  58,  59],
         [ 60,  61,  62,  63,  64],
         [ 65,  66,  67,  68,  69],
         [ 70,  71,  72,  73,  74]],

        [[ 75,  76,  77,  78,  79],
         [ 80,  81,  82,  83,  84],
         [ 85,  86,  87,  88,  89],
         [ 90,  91,  92,  93,  94],
         [ 95,  96,  97,  98,  99]],

        [[100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109],
         [110, 111, 112, 113, 114],
         [115, 116, 117, 118, 119],
         [120, 121, 122, 123, 124]],

        [[125, 126, 127, 128, 129],
         [130, 131, 132, 133, 134],
         [135, 136

(25, 5, 1)

5

In [4]:
torch.diag(torch.tensor(1).repeat(5))

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])

In [5]:
bb = INF(2,5,6)
bb

bb.shape

tensor([[[-inf, -0., -0., -0., -0.],
         [-0., -inf, -0., -0., -0.],
         [-0., -0., -inf, -0., -0.],
         [-0., -0., -0., -inf, -0.],
         [-0., -0., -0., -0., -inf]],

        [[-inf, -0., -0., -0., -0.],
         [-0., -inf, -0., -0., -0.],
         [-0., -0., -inf, -0., -0.],
         [-0., -0., -0., -inf, -0.],
         [-0., -0., -0., -0., -inf]],

        [[-inf, -0., -0., -0., -0.],
         [-0., -inf, -0., -0., -0.],
         [-0., -0., -inf, -0., -0.],
         [-0., -0., -0., -inf, -0.],
         [-0., -0., -0., -0., -inf]],

        [[-inf, -0., -0., -0., -0.],
         [-0., -inf, -0., -0., -0.],
         [-0., -0., -inf, -0., -0.],
         [-0., -0., -0., -inf, -0.],
         [-0., -0., -0., -0., -inf]],

        [[-inf, -0., -0., -0., -0.],
         [-0., -inf, -0., -0., -0.],
         [-0., -0., -inf, -0., -0.],
         [-0., -0., -0., -inf, -0.],
         [-0., -0., -0., -0., -inf]],

        [[-inf, -0., -0., -0., -0.],
         [-0., -inf, -0., -0

torch.Size([12, 5, 5])

In [6]:
cc = (aa.cuda()+bb).cuda().view(2,6,5,5).permute(0,2,1,3);cc

tensor([[[[-inf,   1.,   2.,   3.,   4.],
          [-inf,  26.,  27.,  28.,  29.],
          [-inf,  51.,  52.,  53.,  54.],
          [-inf,  76.,  77.,  78.,  79.],
          [-inf, 101., 102., 103., 104.],
          [-inf, 126., 127., 128., 129.]],

         [[  5., -inf,   7.,   8.,   9.],
          [ 30., -inf,  32.,  33.,  34.],
          [ 55., -inf,  57.,  58.,  59.],
          [ 80., -inf,  82.,  83.,  84.],
          [105., -inf, 107., 108., 109.],
          [130., -inf, 132., 133., 134.]],

         [[ 10.,  11., -inf,  13.,  14.],
          [ 35.,  36., -inf,  38.,  39.],
          [ 60.,  61., -inf,  63.,  64.],
          [ 85.,  86., -inf,  88.,  89.],
          [110., 111., -inf, 113., 114.],
          [135., 136., -inf, 138., 139.]],

         [[ 15.,  16.,  17., -inf,  19.],
          [ 40.,  41.,  42., -inf,  44.],
          [ 65.,  66.,  67., -inf,  69.],
          [ 90.,  91.,  92., -inf,  94.],
          [115., 116., 117., -inf, 119.],
          [140., 141., 142.,

In [7]:
torch.manual_seed(1)
dd = torch.randn(2,5,6,6).cuda()
dd

tensor([[[[-1.5256e+00, -7.5023e-01, -6.5398e-01, -1.6095e+00, -1.0017e-01,
           -6.0919e-01],
          [-9.7977e-01, -1.6091e+00, -7.1214e-01,  3.0372e-01, -7.7731e-01,
           -2.5146e-01],
          [-2.2227e-01,  1.6871e+00,  2.2843e-01,  4.6764e-01, -6.9697e-01,
           -1.1608e+00],
          [ 6.9954e-01,  1.9908e-01,  8.6569e-01,  2.4440e-01, -6.6291e-01,
            8.0731e-01],
          [ 1.1017e+00, -1.7594e-01, -2.2456e+00, -1.4465e+00,  6.1155e-02,
           -6.1774e-01],
          [-7.9807e-01, -1.3162e-01,  1.8793e+00, -7.2132e-02,  1.5777e-01,
           -7.7345e-01]],

         [[ 1.9906e-01,  4.5703e-02,  1.5296e-01, -4.7568e-01, -1.1102e-01,
            2.9274e-01],
          [-1.5785e-01, -2.8787e-02,  2.3571e+00, -1.0373e+00,  1.5748e+00,
           -6.2985e-01],
          [-9.2739e-01,  5.4514e-01,  6.6280e-02, -4.3704e-01,  7.6260e-01,
            4.4151e-01],
          [ 1.1651e+00,  2.0154e+00,  1.3741e-01,  9.3864e-01, -1.8600e-01,
           -6

In [13]:
cc.shape

torch.Size([2, 5, 6, 5])

In [8]:
ee = torch.cat([cc,dd],3)
ee.shape
ee


torch.Size([2, 5, 6, 11])

tensor([[[[       -inf,  1.0000e+00,  2.0000e+00,  3.0000e+00,  4.0000e+00,
           -1.5256e+00, -7.5023e-01, -6.5398e-01, -1.6095e+00, -1.0017e-01,
           -6.0919e-01],
          [       -inf,  2.6000e+01,  2.7000e+01,  2.8000e+01,  2.9000e+01,
           -9.7977e-01, -1.6091e+00, -7.1214e-01,  3.0372e-01, -7.7731e-01,
           -2.5146e-01],
          [       -inf,  5.1000e+01,  5.2000e+01,  5.3000e+01,  5.4000e+01,
           -2.2227e-01,  1.6871e+00,  2.2843e-01,  4.6764e-01, -6.9697e-01,
           -1.1608e+00],
          [       -inf,  7.6000e+01,  7.7000e+01,  7.8000e+01,  7.9000e+01,
            6.9954e-01,  1.9908e-01,  8.6569e-01,  2.4440e-01, -6.6291e-01,
            8.0731e-01],
          [       -inf,  1.0100e+02,  1.0200e+02,  1.0300e+02,  1.0400e+02,
            1.1017e+00, -1.7594e-01, -2.2456e+00, -1.4465e+00,  6.1155e-02,
           -6.1774e-01],
          [       -inf,  1.2600e+02,  1.2700e+02,  1.2800e+02,  1.2900e+02,
           -7.9807e-01, -1.3162e-01,  1

In [9]:
test = torch.Tensor([1,2,3]).repeat(3).reshape(3,3)
test

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.]])

concate = self.softmax(torch.cat([energy_H, energy_W], 3)) computes the attention maps and one overlapped position was set to 0 by the self.INF, so there are H+W-1 non-zero values + 1 zero value.

In [14]:
softmax(ee)

/home/ubuntu/miniconda3/envs/fastai2/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[[[0.0000e+00, 2.0473e-09, 2.0474e-09, 2.0611e-09, 3.0384e-07,
           9.4136e-02, 1.4478e-01, 7.6298e-02, 5.7486e-02, 1.4068e-01,
           1.4787e-01],
          [0.0000e+00, 2.0473e-09, 2.0474e-09, 2.0611e-09, 3.0384e-07,
           4.6530e-02, 4.9740e-02, 3.3694e-02, 2.5644e-01, 5.2072e-02,
           9.9701e-02],
          [0.0000e+00, 2.0473e-09, 2.0474e-09, 2.0611e-09, 3.0384e-07,
           3.9879e-02, 3.6223e-01, 2.5170e-01, 7.1195e-02, 8.9133e-02,
           3.3599e-02],
          [0.0000e+00, 2.0473e-09, 2.0474e-09, 2.0611e-09, 3.0384e-07,
           2.8305e-01, 9.4847e-02, 3.6853e-01, 2.5520e-01, 7.9204e-02,
           1.3933e-01],
          [0.0000e+00, 2.0473e-09, 2.0474e-09, 2.0611e-09, 3.0384e-07,
           2.5699e-01, 9.0260e-02, 1.3878e-02, 1.3686e-01, 1.5405e-01,
           2.8697e-01],
          [0.0000e+00, 2.0473e-09, 2.0474e-09, 2.0611e-09, 3.0384e-07,
           1.0946e-01, 1.2273e-01, 2.7974e-01, 1.0425e-01, 7.9957e-02,
           6.1139e-02]],

  

In [15]:
con1 = [1,2,3]
con2 = [4,5,6]
out = [con1,con2]

In [22]:
# explore the relationship between classes and context information
#  Class Oriented Context Fusion Module
class CC_module(nn.Module):
    def __init__(self,in_dim):
        super(CC_module, self).__init__()
        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.softmax = Softmax(dim=3)
        self.INF = INF
        self.gamma = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        m_batchsize, _, height, width = x.size() # [2, 64, 5, 6] BCHW
        proj_query = self.query_conv(x) # [2, 8, 5, 6]
        proj_query_H = proj_query.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height).permute(0, 2, 1) # [12, 5, 8]
        proj_query_W = proj_query.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width).permute(0, 2, 1) # [10, 6, 8]
        proj_key = self.key_conv(x)# [2, 8, 5, 6]
        proj_key_H = proj_key.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height)   # [12, 8, 5]
        proj_key_W = proj_key.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width) # [10, 8, 6]
        proj_value = self.value_conv(x) # [2, 64, 5, 6]
        proj_value_H = proj_value.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height)   # [12, 64, 5]
        proj_value_W = proj_value.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width)   # [10, 64, 6]
        energy_H = (torch.bmm(proj_query_H, proj_key_H)+self.INF(m_batchsize, height, width)).view(m_batchsize,width,height,height).permute(0,2,1,3)# [12,5,5] -- > [2, 5, 6, 5]
        energy_W = torch.bmm(proj_query_W, proj_key_W).view(m_batchsize,height,width,width) #[2, 5, 6, 6]
        concate = self.softmax(torch.cat([energy_H, energy_W], 3))# [2,5,6,11]

        att_H = concate[:,:,:,0:height].permute(0,2,1,3).contiguous().view(m_batchsize*width,height,height) #[12,5,5]
        #print(concate)
        #print(att_H) 
        att_W = concate[:,:,:,height:height+width].contiguous().view(m_batchsize*height,width,width)# [10, 6, 6]
        out_H = torch.bmm(proj_value_H, att_H.permute(0, 2, 1)).view(m_batchsize,width,-1,height).permute(0,2,3,1) # [2, 64, 5, 6]
        out_W = torch.bmm(proj_value_W, att_W.permute(0, 2, 1)).view(m_batchsize,height,-1,width).permute(0,2,1,3)#[2, 64, 5, 6])
        #print(out_H.size(),out_W.size())
        return self.gamma*(out_H + out_W) + x



if __name__ == '__main__':
    model = CC_module(64)
    # cuda0 = torch.device('cuda:0')
    model.cuda()
    x = torch.randn(2, 64, 5, 6).cuda()
    out = model(x)
    print(out.shape)
    torch.cuda.empty_cache()
# %%


CC_module(
  (query_conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=3)
)

torch.Size([2, 64, 5, 6])


In [ ]:

class _ObjectAttentionBlock(nn.Module):
    '''
    The basic implementation for object context block
    Input:
        N X C X H X W
    Parameters:
        in_channels       : the dimension of the input feature map
        key_channels      : the dimension after the key/query transform
        scale             : choose the scale to downsample the input feature maps (save memory cost)
        bn_type           : specify the bn type
    Return:
        N X C X H X W
    '''
    def __init__(self, 
                 in_channels, 
                 key_channels, 
                 scale=1, 
                 bn_type=None):
        super(_ObjectAttentionBlock, self).__init__()
        self.scale = scale
        self.in_channels = in_channels
        self.key_channels = key_channels
        self.pool = nn.MaxPool2d(kernel_size=(scale, scale))
        self.f_pixel = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
            nn.Conv2d(in_channels=self.key_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
        )
        self.f_object = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
            nn.Conv2d(in_channels=self.key_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
        )
        self.f_down = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=self.key_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.key_channels, bn_type=bn_type),
        )
        self.f_up = nn.Sequential(
            nn.Conv2d(in_channels=self.key_channels, out_channels=self.in_channels,
                kernel_size=1, stride=1, padding=0, bias=False),
            ModuleHelper.BNReLU(self.in_channels, bn_type=bn_type),
        )

    def forward(self, x, proxy):
        # x ([1, 512, 64, 64]) proxy [1, 512, 19, 1]
        batch_size, h, w = x.size(0), x.size(2), x.size(3)
        if self.scale > 1:
            x = self.pool(x)

        query = self.f_pixel(x).view(batch_size, self.key_channels, -1)# key 256
        query = query.permute(0, 2, 1) # (1,4096,256)
        key = self.f_object(proxy).view(batch_size, self.key_channels, -1)# [1, 256, 19])
        value = self.f_down(proxy).view(batch_size, self.key_channels, -1)# 
        value = value.permute(0, 2, 1)# [1, 19, 256]

        sim_map = torch.matmul(query, key)# [1, 4096, 19]
        sim_map = (self.key_channels**-.5) * sim_map # [1, 4096, 19]
        sim_map = F.softmax(sim_map, dim=-1)   # [1, 4096, 19]

        # add bg context ...
        context = torch.matmul(sim_map, value) # [1, 4096, 256]
        context = context.permute(0, 2, 1).contiguous() # [1, 256, 4096]
        context = context.view(batch_size, self.key_channels, *x.size()[2:])#[1, 256, 64, 64] 
        context = self.f_up(context)#[1, 512, 64, 64]
        if self.scale > 1:
            context = F.interpolate(input=context, size=(h, w), mode='bilinear', align_corners=ALIGN_CORNERS)

        return context

In [ ]:
class _CrossAttention(nn.Module):
    def __init__(self,in_dim):
        super(CC_module, self).__init__()
        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.softmax = Softmax(dim=3)
        self.INF = INF
        self.gamma = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        m_batchsize, _, height, width = x.size() # [2, 64, 5, 6] BCHW
        
        proj_query = self.query_conv(x) # [2, 8, 5, 6]
        proj_query_H = proj_query.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height).permute(0, 2, 1) # [12, 5, 8]
        proj_query_W = proj_query.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width).permute(0, 2, 1) # [10, 6, 8]
        
        proj_key = self.key_conv(x)# [2, 8, 5, 6]
        proj_key_H = proj_key.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height)   # [12, 8, 5]
        proj_key_W = proj_key.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width) # [10, 8, 6]
        
        proj_value = self.value_conv(x) # [2, 64, 5, 6]
        proj_value_H = proj_value.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height)   # [12, 64, 5]
        proj_value_W = proj_value.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width)   # [10, 64, 6]
        
        energy_H = (torch.bmm(proj_query_H, proj_key_H)+self.INF(m_batchsize, height, width)).view(m_batchsize,width,height,height).permute(0,2,1,3)# [12,5,5] -- > [2, 5, 6, 5]
        energy_W = torch.bmm(proj_query_W, proj_key_W).view(m_batchsize,height,width,width) #[2, 5, 6, 6]
        concate = self.softmax(torch.cat([energy_H, energy_W], 3))# [2,5,6,11]

        att_H = concate[:,:,:,0:height].permute(0,2,1,3).contiguous().view(m_batchsize*width,height,height) #[12,5,5]
        #print(concate)
        #print(att_H) 
        att_W = concate[:,:,:,height:height+width].contiguous().view(m_batchsize*height,width,width)# [10, 6, 6]
        
        out_H = torch.bmm(proj_value_H, att_H.permute(0, 2, 1)).view(m_batchsize,width,-1,height).permute(0,2,3,1) # [2, 64, 5, 6]
        out_W = torch.bmm(proj_value_W, att_W.permute(0, 2, 1)).view(m_batchsize,height,-1,width).permute(0,2,1,3)#[2, 64, 5, 6])
        
        #print(out_H.size(),out_W.size())
        return self.gamma*(out_H + out_W) + x

In [34]:
test = nn.Parameter(torch.zeros(1))
test
aa = torch.Tensor([[1,2,3],[4,5,6]])
bb = torch.ones(2,3)
aa + bb
(test*aa)
64**2

Parameter containing:
tensor([0.], requires_grad=True)

tensor([[2., 3., 4.],
        [5., 6., 7.]])

tensor([[0., 0., 0.],
        [0., 0., 0.]], grad_fn=<MulBackward0>)

4096

In [37]:
aa = torch.zeros(1)
aa

tensor([0.])